### Importing Libraries

In [72]:
%matplotlib inline
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

import subprocess as sp
import sys,os,shutil,pickle,time
from os.path import join
plt.style.use('default')

### Reading the Data

In [73]:
df= pd.read_csv('Vehicles.csv')

In [74]:
df.shape

(423857, 25)

In [76]:
df = df[['price', 'year', 'manufacturer', 'condition', 'cylinders', 'fuel', 'odometer', 'title_status', 'transmission',
       'drive', 'size', 'type', 'paint_color']]

### Data Cleaning

Let's eliminate extreme values from price column

In [77]:
df=df[(df.price < 100000) & (df.price >= 750 )]
df.shape

(376473, 13)

Odometers higher than 300k and lower than 10, and models earlier than 1985 may not be good observations for a robust prediction:

In [78]:
df=df[(df.odometer < 300000) & (df.odometer > 1000)]
df=df[df.year>1985]
df.shape

(232349, 13)

In [79]:
df['condition'].isnull().sum()

98231

In [80]:
df.loc[(df['year'] >= 2017)]['condition'].isnull().sum()

30461

In [81]:
df.condition.value_counts()

excellent    64937
good         51387
like new     13178
fair          3998
new            391
salvage        227
Name: condition, dtype: int64

In [82]:
df.loc[df.year>=2019, 'condition'] = df.loc[df.year>=2019, 'condition'].fillna('new')
df.loc[df.year>=2017, 'condition'] = df.loc[df.year>=2017, 'condition'].fillna('like new')

In [83]:
df.condition.value_counts()

excellent    64937
good         51387
like new     34405
new           9625
fair          3998
salvage        227
Name: condition, dtype: int64

I will assume that milage of the car affects it's condition. Therefore, I will fill missing values depending on odometer.

In [84]:
excellent_odo_mean = df[df['condition'] == 'excellent']['odometer'].mean()
good_odo_mean = df[df['condition'] == 'good']['odometer'].mean()
like_new_odo_mean = df[df['condition'] == 'like new']['odometer'].mean()
salvage_odo_mean = df[df['condition'] == 'salvage']['odometer'].mean()
fair_odo_mean = df[df['condition'] == 'fair']['odometer'].mean()

In [85]:
print('Like new average odometer:', round( like_new_odo_mean,2))
print('Excellent average odometer:', round( excellent_odo_mean,2))
print('Good average odometer:', round( good_odo_mean,2))
print('Fair average odometer:', round( fair_odo_mean,2))
print('Salvage average odometer:', round( salvage_odo_mean,2))

Like new average odometer: 57093.17
Excellent average odometer: 106599.31
Good average odometer: 109782.47
Fair average odometer: 173469.2
Salvage average odometer: 140622.37


In [86]:
df.loc[df['odometer'] <= like_new_odo_mean, 'condition'].value_counts()

like new     21646
good         13343
excellent    12677
new           9280
fair            88
salvage         25
Name: condition, dtype: int64

In [87]:
df.loc[df['odometer'] <= like_new_odo_mean, 'condition'] = df.loc[df['odometer'] <= like_new_odo_mean, 'condition'].fillna('like new')

df.loc[df['odometer'] >= fair_odo_mean, 'condition'] = df.loc[df['odometer'] >= fair_odo_mean, 'condition'].fillna('fair')

df.loc[((df['odometer'] > like_new_odo_mean) & 
       (df['odometer'] <= excellent_odo_mean)), 'condition'] = df.loc[((df['odometer'] > like_new_odo_mean) & 
       (df['odometer'] <= excellent_odo_mean)), 'condition'].fillna('excellent')

df.loc[((df['odometer'] > excellent_odo_mean) & 
       (df['odometer'] <= good_odo_mean)), 'condition'] = df.loc[((df['odometer'] > excellent_odo_mean) & 
       (df['odometer'] <= good_odo_mean)), 'condition'].fillna('good')

df.loc[((df['odometer'] > good_odo_mean) & 
       (df['odometer'] <= fair_odo_mean)), 'condition'] = df.loc[((df['odometer'] > good_odo_mean) & 
       (df['odometer'] <= fair_odo_mean)), 'condition'].fillna('salvage')

In [88]:
df.cylinders.value_counts()

4 cylinders     56064
6 cylinders     55438
8 cylinders     38391
5 cylinders      1329
10 cylinders      619
other             446
3 cylinders       313
12 cylinders      124
Name: cylinders, dtype: int64

In [89]:
df['transmission'] = df['transmission'].fillna(method='ffill')
df['cylinders'] = df['cylinders'].fillna(method='ffill')
df['title_status'] = df['title_status'].fillna(method='ffill')
df['fuel'] = df['fuel'].fillna(method='ffill')
df['size'] = df['size'].fillna(method='ffill')
df['paint_color'] = df['paint_color'].fillna(method='ffill')
df['drive'] = df['drive'].fillna(method='ffill')
df['manufacturer'] = df['manufacturer'].fillna(method='ffill')
df['type'] = df['type'].fillna(method='ffill')

In [90]:
df.isnull().sum()

price           0
year            0
manufacturer    0
condition       0
cylinders       0
fuel            0
odometer        0
title_status    0
transmission    0
drive           0
size            3
type            3
paint_color     0
dtype: int64

In [91]:
df.sample(10)

,price,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color
88633,2500,2007.0,volkswagen,good,5 cylinders,gas,255000.0,clean,automatic,fwd,compact,hatchback,grey
405854,66500,2018.0,ram,like new,6 cylinders,diesel,9000.0,lien,manual,4wd,full-size,truck,red
318114,12900,2009.0,mercedes-benz,excellent,5 cylinders,gas,117039.0,clean,automatic,fwd,full-size,SUV,black
70372,17995,2003.0,lexus,excellent,8 cylinders,gas,39657.0,clean,automatic,rwd,compact,sedan,blue
394894,9995,2008.0,toyota,excellent,4 cylinders,gas,142649.0,clean,automatic,4wd,compact,SUV,silver
318703,21995,2019.0,chrysler,new,6 cylinders,gas,34217.0,clean,automatic,rwd,full-size,sedan,black
9681,15995,2017.0,ford,like new,4 cylinders,gas,77843.0,clean,automatic,4wd,compact,van,white
292023,14000,2017.0,hyundai,like new,4 cylinders,gas,40941.0,clean,automatic,fwd,compact,other,silver
1478,14950,2013.0,ram,good,6 cylinders,gas,126015.0,clean,automatic,rwd,full-size,pickup,white
420375,25900,1998.0,chevrolet,like new,6 cylinders,gas,17752.0,clean,automatic,fwd,sub-compact,SUV,white


In [92]:
import seaborn as sns
import matplotlib.pyplot as plt
from scipy import stats

In [94]:
df.set_index(['price', 'year']).sample(5)

,,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color
price,year,,,,,,,,,,,
41895,2012.0,honda,like new,6 cylinders,gas,28593.0,clean,automatic,fwd,sub-compact,coupe,brown
16000,2006.0,lincoln,excellent,8 cylinders,gas,103885.0,clean,automatic,rwd,full-size,sedan,white
12800,2010.0,buick,excellent,6 cylinders,gas,93000.0,clean,automatic,4wd,full-size,SUV,white
15995,2016.0,jeep,excellent,4 cylinders,gas,80273.0,clean,automatic,4wd,compact,SUV,white
67999,2019.0,ford,new,8 cylinders,diesel,20000.0,clean,automatic,rwd,full-size,SUV,black


In [95]:
df.sort_values('year').sample(10)

,price,year,manufacturer,condition,cylinders,fuel,odometer,title_status,transmission,drive,size,type,paint_color
382296,16995,2001.0,ford,good,8 cylinders,gas,149680.0,clean,automatic,4wd,full-size,truck,red
297637,12400,2015.0,bmw,good,4 cylinders,gas,98239.0,clean,automatic,4wd,full-size,wagon,blue
87460,6900,2011.0,lincoln,excellent,6 cylinders,gas,127263.0,clean,automatic,4wd,compact,sedan,silver
229348,2200,2002.0,mercedes-benz,good,8 cylinders,gas,179000.0,clean,automatic,4wd,full-size,SUV,silver
267346,47758,2017.0,ford,like new,6 cylinders,gas,86302.0,clean,automatic,fwd,sub-compact,convertible,silver
142980,36488,2016.0,gmc,fair,6 cylinders,diesel,178299.0,clean,automatic,4wd,mid-size,pickup,red
135041,22900,2013.0,chevrolet,good,8 cylinders,diesel,210000.0,clean,automatic,4wd,full-size,truck,white
416989,5995,2010.0,gmc,fair,4 cylinders,gas,177353.0,clean,automatic,4wd,mid-size,pickup,black
206266,10990,2010.0,mercedes-benz,salvage,4 cylinders,gas,144692.0,clean,automatic,fwd,compact,SUV,white
372551,22995,2017.0,ford,like new,6 cylinders,gas,7618.0,clean,automatic,4wd,mid-size,SUV,blue


In [97]:
df = df.dropna()

In [98]:
df.to_csv('cars_cleaned.csv', index=False)